In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../cities.csv")

# Display sample data
city_data_df.head()

City_ID               City      Lat       Lng  Max Temp  Humidity  \
0        0               roma  43.2128  -75.4557    270.86        89   
1        1            margate  51.3813    1.3862    284.76        95   
2        2         queenstown -31.8976   26.8753    282.20        86   
3        3  port-aux-francais -49.3500   70.2167    277.59        79   
4        4         wellington -41.2866  174.7756    290.96        83   

   Cloudiness  Wind Speed Country        Date  
0           0        0.00      US  1699846747  
1         100        2.24      GB  1699846906  
2           2        0.10      ZA  1699846906  
3         100        2.90      TF  1699846906  
4          75       13.38      NZ  1699846750

In [3]:
%%capture --no-display

# Create a new DataFrame with unique cities and their coordinates
unique_cities_df = city_data_df[['City', 'Lat', 'Lng', 'Humidity']].drop_duplicates()

# Plot the DataFrame with hvplot
map_plot = unique_cities_df.hvplot.points(
    x='Lng',
    y='Lat',
    c='Humidity',
    geo= True,
    hover_cols=['City', 'Humidity'],
    size='Humidity',
    title='City Map with Humidity',
    frame_width=700,
    tiles='CartoLight',
)

# Display the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

In [4]:
 # Narrow down cities that fit criteria and drop any results with null values
criteria_df = city_data_df[(city_data_df['Humidity'] < 50) & (city_data_df['Max Temp'] >= 20)]

# Drop any rows with null values
cleaned_data_df = criteria_df.dropna()

# Display sample data
cleaned_data_df.head()


City_ID           City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
6         6  pico truncado -46.7949  -67.9573    282.40        41           0   
11       11          uyuni -20.4597  -66.8250    284.32        32          61   
23       23         newman  37.3138 -121.0208    290.34        39          96   
28       28         disina  11.4814    9.9190    293.58        23          61   
41       41  blackmans bay -43.0167  147.3167    294.22        35           2   

    Wind Speed Country        Date  
6        10.43      AR  1699846907  
11        4.34      BO  1699846907  
23        2.42      US  1699846910  
28        3.09      NG  1699846910  
41        2.35      AU  1699846912

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df
hotel_df = cleaned_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

City Country      Lat       Lng  Humidity Hotel Name
6   pico truncado      AR -46.7949  -67.9573        41           
11          uyuni      BO -20.4597  -66.8250        32           
23         newman      US  37.3138 -121.0208        39           
28         disina      NG  11.4814    9.9190        23           
41  blackmans bay      AU -43.0167  147.3167        35

In [6]:
# Set parameters to search for a hotel
radius = 10000  
params = {
    'categories': 'accommodation.hotel',
    'limit': 20,  
    'apiKey': geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
# Display sample data
hotel_df


Starting hotel search
pico truncado - nearest hotel: No hotel found
uyuni - nearest hotel: Hostel Jerian
newman - nearest hotel: No hotel found
disina - nearest hotel: No hotel found
blackmans bay - nearest hotel: Villa Howden
serdar - nearest hotel: Chynar Hotel
new norfolk - nearest hotel: Woodbridge on the Derwent
margaret river - nearest hotel: Margaret River Hotel
zavodskoy - nearest hotel: No hotel found
as sulayyil - nearest hotel: No hotel found
rawson - nearest hotel: Hotel Deportivo
zhob - nearest hotel: No hotel found
g'azalkent - nearest hotel: Semurg' oromgoxi
bhadrakh - nearest hotel: No hotel found
nalut - nearest hotel: No hotel found
leo - nearest hotel: Auberge le prestige
yangquan - nearest hotel: 7天酒店
tchintabaraden - nearest hotel: No hotel found
al jawf - nearest hotel: No hotel found
norak - nearest hotel: Sayora.S Hotel
pampierstad - nearest hotel: No hotel found
behshahr - nearest hotel: هتل اشرف
agadez - nearest hotel: No hotel found
puerto madryn - nearest ho

City Country      Lat       Lng  Humidity      Hotel Name
6    pico truncado      AR -46.7949  -67.9573        41  No hotel found
11           uyuni      BO -20.4597  -66.8250        32   Hostel Jerian
23          newman      US  37.3138 -121.0208        39  No hotel found
28          disina      NG  11.4814    9.9190        23  No hotel found
41   blackmans bay      AU -43.0167  147.3167        35    Villa Howden
..             ...     ...      ...       ...       ...             ...
553          wabag      PG  -5.4781  143.7021        47  No hotel found
556         viedma      AR -40.8135  -62.9967        19           Níjar
558           bima      ID  -8.4667  118.7167        48  No hotel found
563        bomdila      IN  27.2500   92.4000        42         Seagull
567            soe      ID  -9.8607  124.2840        30            DENA

[92 rows x 6 columns]

In [8]:
# Configure the map plot
map_plot_with_hotel = hotel_df.hvplot.points(
    x='Lng',
    y='Lat',
    c='Humidity',
    geo= True,
    hover_cols=['City', 'Country', 'Humidity', 'Hotel Name'], 
    size='Humidity',
    title='City Map with Humidity and Hotel Information',
    frame_width=700,
    tiles='CartoLight', 
)
# Display the map
map_plot_with_hotel


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)